In [1]:
!pip install -q nltk datasets transformers torch sentencepiece sentence_transformers
!pip install evaluate rouge rouge_score

In [2]:
!pip install -q datasets nltk spacy
!python -m spacy download en_core_web_trf
!pip install cupy-cuda12x

Traceback (most recent call last):
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/runpy.py", line 196, in _run_module_as_main
    return _run_code(code, main_globals, None,
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/runpy.py", line 86, in _run_code
    exec(code, run_globals)
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/spacy/__main__.py", line 4, in <module>
    setup_cli()
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/spacy/cli/_util.py", line 68, in setup_cli
    registry.cli.get_all()
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/catalogue/__init__.py", line 110, in get_all
    result.update(self.get_entry_points())
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/catalogue/__init__.py", line 125, in get_entry_points
    result[entry_point.name] = entry_point.load()
  File "/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/importlib/me

In [4]:

import nltk
from nltk.tokenize import sent_tokenize
from datasets import load_dataset, DatasetDict,Dataset
from transformers import AutoTokenizer, AutoModelForCausalLM, pipeline
from sentence_transformers import SentenceTransformer, util
import torch
import re
from tqdm import tqdm
# from evaluate import load
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer
import numpy as np
# rouge = load("rouge")
nltk.download("punkt")
nltk.download('punkt_tab')

/home/zeus/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/io/image.py:13: UserWarning: Failed to load image Python extension: /system/conda/miniconda3/envs/cloudspace/lib/python3.10/site-packages/torchvision/image.so: undefined symbol: _ZN3c104cuda20CUDACachingAllocator12recordStreamERKNS_7DataPtrENS0_10CUDAStreamE
  warn(f"Failed to load image Python extension: {e}")
[nltk_data] Downloading package punkt to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to
[nltk_data]     /teamspace/studios/this_studio/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!


True

In [5]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(device)

cuda


In [6]:

def clean_summary_text(text):
    """
    Pre-clean the summary text to fix artifacts such as a conjunction
    immediately followed by a period. For example, "John and . Audrey" becomes
    "John and Audrey".
    """
    # Remove a period if it follows a conjunction with only whitespace in between.
    text = re.sub(r'\b(and|or|but)\s*\.\s*', r'\1 ', text, flags=re.IGNORECASE)
    return text


In [7]:

def tokens_length(text):
    """
    Dummy token count: assume each word is a token.
    """
    return len(text.split())

In [8]:

def merge_incomplete_sentences(sentences, min_tokens=10):
    """
    Merge summary sentences that appear incomplete.
    
    Heuristic:
      - If a sentence has fewer than `min_tokens` tokens,
      - Or if it ends with a conjunction (e.g. "and", "or", "but") possibly
        followed by punctuation,
    then remove trailing punctuation and merge it with the following sentence.
    
    Returns a new list of sentences.
    """
    merged = []
    i = 0
    while i < len(sentences):
        current = sentences[i].strip()
        # Remove extra punctuation at the end if it looks like an artifact.
        current = re.sub(r'\s*[,\.;:]+\s*$', '', current)
        tokens = current.split()
        # If not the last sentence and the sentence is very short or ends with a conjunction...
        if i < len(sentences) - 1 and (len(tokens) < min_tokens or re.search(r'\b(and|or|but)$', current, re.IGNORECASE)):
            # Merge current with the next sentence (and remove any extra punctuation).
            next_sent = re.sub(r'^\s*[,\.;:]+\s*', '', sentences[i+1].strip())
            merged.append(current + " " + next_sent)
            i += 2
        else:
            merged.append(current)
            i += 1
    return merged

In [9]:

def normalize_person_name(name):
    """
    Normalize a person name so that variants map to the same key.
    For example, "Harry Potter" and "H. Potter" will both be converted to "h_potter".
    
    The heuristic:
      - Remove punctuation,
      - Lowercase the text,
      - Split into tokens,
      - If two or more tokens exist, return: first letter of the first token + '_' + last token.
      - Otherwise, return the token.
    """
    name_clean = re.sub(r'[^\w\s]', '', name).strip().lower()
    tokens = name_clean.split()
    if not tokens:
        return ""
    if len(tokens) == 1:
        return tokens[0]
    return tokens[0][0] + "_" + tokens[-1]

In [10]:

def anonymize_names_in_text(text, mapping, next_person_index):
    """
    Replace person names in the text with placeholders like "person_1".
    
    Uses spaCy to detect PERSON entities. Contiguous PERSON entities (e.g.,
    "Albert and Connon") are merged into one entity. Each unique normalized person
    name is replaced with a placeholder.
    
    Parameters:
      - text: The input text.
      - mapping: A dict mapping normalized names to placeholders.
      - next_person_index: The next available placeholder index.
    
    Returns:
      - anonymized_text: The text after replacement.
      - mapping: The updated mapping.
      - next_person_index: The updated next available index.
    """
    doc = nlp(text)
    person_entities = []
    for ent in doc.ents:
        if ent.label_ == "PERSON":
            person_entities.append((ent.start_char, ent.end_char, ent.text))
    
    # Merge contiguous PERSON entities.
    merged_entities = []
    if person_entities:
        current_start, current_end, current_text = person_entities[0]
        for start, end, ent_text in person_entities[1:]:
            gap = text[current_end:start]
            if re.match(r'^\s*(and|,|&)?\s*$', gap):
                current_end = end
                current_text += " " + ent_text
            else:
                merged_entities.append((current_start, current_end, current_text))
                current_start, current_end, current_text = start, end, ent_text
        merged_entities.append((current_start, current_end, current_text))
    
    anonymized_text = ""
    last_index = 0
    for start, end, name in merged_entities:
        anonymized_text += text[last_index:start]
        norm = normalize_person_name(name)
        if norm not in mapping:
            mapping[norm] = f"person_{next_person_index}"
            next_person_index += 1
        anonymized_text += mapping[norm]
        last_index = end
    anonymized_text += text[last_index:]
    return anonymized_text, mapping, next_person_index

In [11]:

def merge_identical_article_chunks(examples):
    """
    Merge examples that have identical article chunks by concatenating their summary sentences.
    
    Parameters:
      - examples: A list of dictionaries with keys "article", "highlight", and "id".
      
    Returns:
      - A new list of examples where examples with identical article chunks have been merged.
    """
    merged = {}
    for ex in examples:
        article = ex["article"]
        if article not in merged:
            merged[article] = {
                "article": article,
                "highlights": ex["highlights"],
                "id": ex["id"]
            }
        else:
            merged[article]["highlights"] += " " + ex["highlights"]
    return list(merged.values())


In [12]:
def process_row_tfidf(example, sim_threshold=0.3, max_total_tokens=400):
    """
    Process one input row (a dict with keys "article" and "highlights")
    and output one or more rows mapping each summary sentence to a matching
    article chunk using TF-IDF cosine similarity.
    
    For each summary sentence:
      1. Identify the best matching article sentence (A) via cosine similarity.
      2. Append the following sentence (B), if available. If (A+B) exceeds the allowed token count,
         then truncate the end of B so that A+B fits exactly.
      3. If (A+B) is shorter than the allowed token count and a sentence before A (C) exists,
         prepend C. If adding C makes the candidate exceed the limit, then truncate tokens from the beginning of C.
      4. Discard the pair if the resulting candidate article chunk or summary is empty.
    
    Parameters:
      - example (dict): Must contain keys "article" and "highlights".
      - sim_threshold (float): (Not used in candidate selection here.)
      - max_total_tokens (int): Maximum allowed token (word) count for article + summary.
    
    Returns:
      - List of dictionaries, each with keys "article_chunk" and "summary_chunk".
        Pairs with empty chunks are discarded.
    """
    import numpy as np
    from sklearn.feature_extraction.text import TfidfVectorizer
    from sklearn.metrics.pairwise import cosine_similarity
    from nltk.tokenize import sent_tokenize

    # Retrieve text from the example.
    article_text = example.get("article", "")
    summary_text = example.get("highlights", "")
    
    # Sentence tokenization.
    article_sents = sent_tokenize(article_text)
    summary_sents = sent_tokenize(summary_text)
    
    if not article_sents or not summary_sents:
        return []
    
    # Precompute tokenized versions and lengths for article sentences.
    article_tokens = [sent.split() for sent in article_sents]
    article_token_lens = [len(tokens) for tokens in article_tokens]
    
    # Precompute tokenized versions for summary sentences.
    summary_tokens_list = [sent.split() for sent in summary_sents]
    
    # Fit the TF-IDF vectorizer on the article sentences.
    vectorizer = TfidfVectorizer()
    vectorizer.fit(article_sents)
    
    # Compute TF-IDF vectors for article sentences.
    article_vectors = vectorizer.transform(article_sents)
    
    # Batch transform all summary sentences.
    summary_vectors = vectorizer.transform(summary_sents)
    # Compute similarity matrix: each row corresponds to a summary sentence.
    similarity_matrix = cosine_similarity(summary_vectors, article_vectors)
    
    output_rows = []
    for i, summ in enumerate(summary_sents):
        # Skip empty summary.
        if not summ.strip():
            continue

        summ_tokens = summary_tokens_list[i]
        len_summ = len(summ_tokens)
        allowed_tokens = max_total_tokens - len_summ
        if allowed_tokens <= 0:
            continue  # Cannot form candidate if summary already exceeds max_total_tokens.
        
        # Step 1: Find best matching article sentence A.
        sims = similarity_matrix[i]
        best_idx = int(np.argmax(sims))
        A = article_sents[best_idx]
        tokens_A = article_tokens[best_idx]
        
        # Initialize candidate with A.
        candidate_tokens = tokens_A[:]
        candidate_text = A
        
        # Step 2: Append following sentence B if available.
        if best_idx < len(article_sents) - 1:
            B = article_sents[best_idx + 1]
            tokens_B = article_tokens[best_idx + 1]
            # Check if full A+B fits.
            if len(candidate_tokens) + len(tokens_B) <= allowed_tokens:
                candidate_tokens += tokens_B
                candidate_text = A + " " + B
            else:
                # Only add as many tokens from B as possible.
                allowed_for_B = allowed_tokens - len(candidate_tokens)
                if allowed_for_B > 0:
                    truncated_B_tokens = tokens_B[:allowed_for_B]
                    candidate_tokens += truncated_B_tokens
                    candidate_text = A + " " + " ".join(truncated_B_tokens)
                # Else, no tokens from B can be added.
        
        # Step 3: If candidate is still shorter than allowed and there's a preceding sentence C.
        if len(candidate_tokens) < allowed_tokens and best_idx > 0:
            C = article_sents[best_idx - 1]
            tokens_C = article_tokens[best_idx - 1]
            # Prepend entire C.
            candidate_with_C_tokens = tokens_C + candidate_tokens
            if len(candidate_with_C_tokens) <= allowed_tokens:
                candidate_tokens = candidate_with_C_tokens
                candidate_text = C + " " + candidate_text
            else:
                # Determine how many tokens from C can be added.
                allowed_for_C = allowed_tokens - len(candidate_tokens)
                if allowed_for_C > 0:
                    # Truncate C from its beginning (i.e. drop the earliest tokens)
                    truncated_C_tokens = tokens_C[-allowed_for_C:]
                    candidate_tokens = truncated_C_tokens + candidate_tokens
                    candidate_text = " ".join(truncated_C_tokens) + " " + candidate_text
                # Else: cannot add any tokens from C.
        
        # Final check: if candidate or summary is empty, skip this pair.
        if not candidate_text.strip() or not summ.strip():
            continue
        
        output_rows.append({
            "article_chunk": candidate_text,
            "summary_chunk": summ
        })
    
    return output_rows


In [14]:
dataset_train = load_dataset("cnn_dailymail", "3.0.0", split="train")
# Process each example with a tqdm progress bar.
processed_examples = []
# subset = dataset_train.select(range(100))
for example in tqdm(dataset_train, desc="Processing examples"):
    # Each example may generate one or more output rows.
    new_rows = process_row_tfidf(example, sim_threshold=0.2, max_total_tokens=400)
    processed_examples.extend(new_rows)

# Create a new Hugging Face dataset from the processed examples.
new_dataset = Dataset.from_list(processed_examples)








Processing examples: 100%|██████████| 287113/287113 [27:30<00:00, 174.00it/s]


In [15]:
new_dataset.save_to_disk("chunked_train")

Saving the dataset (0/1 shards):   0%|          | 0/1057334 [00:00<?, ? examples/s]

In [16]:
new_dataset[999]

{'article_chunk': 'The four soldiers face minimum mandatory sentences of 10 years in prison each for the drug conspiracy and an additional five years, consecutive, for the weapons allegation. The Army Rangers are an elite light infantry fighting force capable of deploying anywhere in the world within 18 hours. They became a permanent presence in the U.S. military in the 1970s.',
 'summary_chunk': 'The Army Rangers are an elite light infantry fighting force .'}

In [14]:
dataset_train = load_dataset("cnn_dailymail", "3.0.0", split="test")
# Process each example with a tqdm progress bar.
processed_examples = []
for example in tqdm(dataset_train, desc="Processing examples"):
    # Each example may generate one or more output rows.
    new_rows = process_row_tfidf(example, sim_threshold=0.2, max_total_tokens=400)
    processed_examples.extend(new_rows)

# Create a new Hugging Face dataset from the processed examples.
new_dataset = Dataset.from_list(processed_examples)
new_dataset.save_to_disk("chunked_test") 







Processing examples: 100%|██████████| 11490/11490 [00:44<00:00, 258.32it/s]


Saving the dataset (0/1 shards):   0%|          | 0/43560 [00:00<?, ? examples/s]

In [15]:
dataset_train = load_dataset("cnn_dailymail", "3.0.0", split="validation")
# Process each example with a tqdm progress bar.
processed_examples = []
for example in tqdm(dataset_train, desc="Processing examples"):
    # Each example may generate one or more output rows.
    new_rows = process_row_tfidf(example, sim_threshold=0.2, max_total_tokens=400)
    processed_examples.extend(new_rows)

# Create a new Hugging Face dataset from the processed examples.
new_dataset = Dataset.from_list(processed_examples)
new_dataset.save_to_disk("chunked_validation") 







Processing examples: 100%|██████████| 13368/13368 [00:51<00:00, 258.52it/s]


Saving the dataset (0/1 shards):   0%|          | 0/53480 [00:00<?, ? examples/s]